# Day 8 — Imputing `Wind_Speed(mph)` and `Weather_Condition`


In [1]:
# Step — Imports & load dataset
import pandas as pd
import numpy as np

dataset_path = r"D:\Infosys SpringBoard Virtual Internship 6.0\US_Accidents_March23.csv"
us_accidents_df = pd.read_csv(dataset_path)
print("Loaded:", us_accidents_df.shape)


Loaded: (7728394, 46)


## Step 1 — Quick check of missing values (before)


In [2]:
# show missing counts and percentages for the two columns we target + top few others
cols = ['Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition']
missing_before = us_accidents_df[cols].isnull().sum()
missing_pct_before = (us_accidents_df[cols].isnull().mean() * 100).round(3)
print("Missing counts (before):\n", missing_before)
print("\nMissing % (before):\n", missing_pct_before)


Missing counts (before):
 Wind_Speed(mph)       571233
Precipitation(in)    2203586
Weather_Condition     173459
dtype: int64

Missing % (before):
 Wind_Speed(mph)       7.391
Precipitation(in)    28.513
Weather_Condition     2.244
dtype: float64


## Step 2 — (RECOMMENDED for beginners) Simple & safe imputation
- `Wind_Speed(mph)` → fill with **median** (robust to outliers).  
- `Weather_Condition` → fill with **state-wise mode** (if available), otherwise global mode.


In [3]:
# ensure Start_Time -> Hour exists (useful later)
us_accidents_df['Start_Time'] = pd.to_datetime(us_accidents_df['Start_Time'], errors='coerce')
if 'Hour' not in us_accidents_df.columns:
    us_accidents_df['Hour'] = us_accidents_df['Start_Time'].dt.hour

# 1) Wind_Speed: fill with median
wind_median = us_accidents_df['Wind_Speed(mph)'].median()
us_accidents_df['Wind_Speed(mph)'] = us_accidents_df['Wind_Speed(mph)'].fillna(wind_median)

# 2) Weather_Condition: try state-wise mode first, then global mode
# compute mode per state
state_mode = us_accidents_df.groupby('State')['Weather_Condition'] \
            .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)

# map state mode to rows and fill
us_accidents_df['Weather_Condition'] = us_accidents_df['Weather_Condition'].fillna(
    us_accidents_df['State'].map(state_mode)
)

# fill any remaining with global mode
global_mode = us_accidents_df['Weather_Condition'].mode().iloc[0]
us_accidents_df['Weather_Condition'] = us_accidents_df['Weather_Condition'].fillna(global_mode)

# Confirm missing after simple imputation
missing_after_simple = us_accidents_df[['Wind_Speed(mph)', 'Weather_Condition']].isnull().sum()
print("\nMissing after simple imputation:\n", missing_after_simple)



Missing after simple imputation:
 Wind_Speed(mph)      0
Weather_Condition    0
dtype: int64


## Step 3 — Smarter, scalable numeric imputation (group medians)
If you want better numeric estimates without heavy ML, use group medians (e.g., median `Wind_Speed` per `State` and `Hour`), then fall back to overall median.


In [4]:
# create a copy (safety) if you want to compare methods
df_group_imp = us_accidents_df.copy()

# compute median wind speed by State + Hour
group_median = df_group_imp.groupby(['State', 'Hour'])['Wind_Speed(mph)'].median()

# fill missing using the group median via map on (State,Hour)
# create a MultiIndex series for mapping
group_median = group_median.rename('grp_med')
df_idx = df_group_imp.set_index(['State','Hour'])
df_idx['Wind_Speed_imp'] = df_idx['Wind_Speed(mph)'].fillna(group_median)
df_group_imp = df_idx.reset_index()

# any remaining missing -> fill with overall median
overall_median = df_group_imp['Wind_Speed(mph)'].median()
df_group_imp['Wind_Speed_imp'] = df_group_imp['Wind_Speed_imp'].fillna(overall_median)

# show change counts
print("Missing Wind_Speed before:", us_accidents_df['Wind_Speed(mph)'].isnull().sum())
print("Missing Wind_Speed after group-median imputation (Wind_Speed_imp):",
      df_group_imp['Wind_Speed_imp'].isnull().sum())


Missing Wind_Speed before: 0
Missing Wind_Speed after group-median imputation (Wind_Speed_imp): 0


## Step 4 — OPTIONAL: KNN imputation on a sample (advanced / slower)
- Use when you want neighbor-based estimates.
- **Requires** scikit-learn (`pip install scikit-learn`) and may be slow for large samples.
- We'll run KNNImputer on a sampled subset, then write imputed Wind_Speed back for those sampled rows.


In [5]:
# If scikit-learn isn't installed uncomment and run this in a cell:
# !pip install scikit-learn

from sklearn.impute import KNNImputer

# Select numeric features that help predict Wind_Speed
num_features = ['Wind_Speed(mph)', 'Temperature(F)', 'Visibility(mi)', 'Pressure(in)', 'Humidity(%)', 'Distance(mi)']

# sample rows (choose sample size you can handle; 50k-100k is reasonable if memory allows)
sample_size = 80000
sample_df = us_accidents_df[num_features].sample(n=sample_size, random_state=42)

# fit KNN imputer on the sample
knn_imputer = KNNImputer(n_neighbors=5)
sample_imputed = knn_imputer.fit_transform(sample_df)

# convert back to DataFrame and restore index so we can place results back
sample_imputed_df = pd.DataFrame(sample_imputed, columns=sample_df.columns, index=sample_df.index)

# put imputed Wind_Speed into main df for sampled rows only
us_accidents_df.loc[sample_imputed_df.index, 'Wind_Speed(mph)'] = sample_imputed_df['Wind_Speed(mph)']

print("After KNN-sample imputation, missing Wind_Speed:", us_accidents_df['Wind_Speed(mph)'].isnull().sum())


After KNN-sample imputation, missing Wind_Speed: 0


## Step 5 — Confirm results, compare methods, and save


In [6]:
# Summary of missing for our columns after whatever steps you applied
print("Final missing counts:")
print(us_accidents_df[['Wind_Speed(mph)', 'Weather_Condition']].isnull().sum())

# If you used df_group_imp and want to keep its Wind_Speed_imp column as final:
# us_accidents_df['Wind_Speed(mph)'] = df_group_imp['Wind_Speed_imp']  # only if indices align; otherwise merge carefully

# Save cleaned smaller sample or the full cleaned dataset (optional)
# us_accidents_df.to_csv(r"D:\Infosys SpringBoard Virtual Internship 6.0\US_Accidents_March23_imputed.csv", index=False)
print("\nDone — inspect the printed counts above. Save the DataFrame after verifying.")


Final missing counts:
Wind_Speed(mph)      0
Weather_Condition    0
dtype: int64

Done — inspect the printed counts above. Save the DataFrame after verifying.
